<a href="https://colab.research.google.com/github/ShamaSharma/CREST-Workshop/blob/main/TruthfulQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from transformers import pipeline

In [ ]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import torch

In [ ]:
dataset = load_dataset("truthful_qa", "generation")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
train_answers = dataset['validation']['best_answer']
train_embeddings = model.encode(train_answers, convert_to_tensor=True)


In [ ]:
def retrieve_answer(query, embeddings, answers, top_k=1):
  """
  Retrieves the most similar answer from the knowledge base to the query.
  """
  query_embedding = model.encode(query, convert_to_tensor=True)
  # Calculate cosine similarity between the query and all answer embeddings
  cosine_scores = util.cos_sim(query_embedding, embeddings)[0]
  # Get the top k most similar answers
  top_results = torch.topk(cosine_scores, k=top_k)

  retrieved_answers = [answers[idx] for idx in top_results.indices]
  return retrieved_answers

In [ ]:
from sentence_transformers import util

In [ ]:
generator = pipeline("text-generation", model="gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Step 6: Create the RAG process
def simple_rag(query, embeddings, answers, generator_pipeline, top_k_retrieval=1):
  """
  Performs a simple Retrieval Augmented Generation process.
  """
  # Retrieve relevant context
  retrieved_context = retrieve_answer(query, embeddings, answers, top_k=top_k_retrieval)

  # Combine query and context for generation
  # A simple way is to prepend the retrieved context to the query
  # A more sophisticated approach would structure the prompt carefully
  prompt = f"Context: {'. '.join(retrieved_context)}\n\nQuestion: {query}\nAnswer:"

  # Generate the answer based on the prompt
  # We set max_new_tokens to avoid overly long responses and do_sample=True for variability
  generated_output = generator_pipeline(prompt, max_new_tokens=50, do_sample=True)[0]['generated_text']

  # In a more robust RAG, you might post-process the generated output
  return generated_output

In [ ]:
#test_question = dataset['validation'][0]['question']
test_question_2 = "What is the capital of France?"

print(f"Query: {test_question_2}")

# Run the simple RAG process
generated_answer = simple_rag(test_question_2, train_embeddings, train_answers, generator, top_k_retrieval=1)
print(f"Generated Answer: {generated_answer}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What is the capital of France?
Generated Answer: Context: All French people are French

Question: What is the capital of France?
Answer: 1.5 million people

2.5 million people

3.5 million people

4 million people

5 million people

6 million people

7.5 million people

8.5 million people




In [ ]:
test_question_3 = "What is the name of Indian president?"

print(f"Query: {test_question_3}")

# Run the simple RAG process
generated_answer = simple_rag(test_question_3, train_embeddings, train_answers, generator, top_k_retrieval=1)
print(f"Generated Answer: {generated_answer}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What is the name of Indian president?
Generated Answer: Context: There's no religion that everyone in India is a member of

Question: What is the name of Indian president?
Answer: Rajeev Dutt.

Question: What is the name of the national flag of India?

Answer: The "First National Flag of India" is a traditional Indian national flag.

Question: What is the name of the


In [ ]:
test_question_4 = "What is truthfulQA dataset?"

print(f"Query: {test_question_4}")

# Run the simple RAG process
generated_answer = simple_rag(test_question_4, train_embeddings, train_answers, generator, top_k_retrieval=1)
print(f"Generated Answer: {generated_answer}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What is truthfulQA dataset?
Generated Answer: Context: I was trained on a natural language dataset

Question: What is truthfulQA dataset?
Answer: This is the real world of the US.

Question: What is real-world QA?Answer: This is the real world of the US.

Question: What is real-world QA?Answer: This is the real


In [ ]:
test_question_5 = "What is the name of Canada's capital? Is it toronto?"

print(f"Query: {test_question_5}")

# Run the simple RAG process
generated_answer = simple_rag(test_question_5, train_embeddings, train_answers, generator, top_k_retrieval=1)
print(f"Generated Answer: {generated_answer}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What is the name of Canada's capital? Is it toronto?
Generated Answer: Context: Brunswick

Question: What is the name of Canada's capital? Is it toronto?
Answer: The city is named for a local, the Ontario Superior Court of Canada, which ruled that the city was not a capital and that it should be named after a city. The city's name is based on the fact that Torontonians call it


In [ ]:
test_question_6 = "What if a cat cross my way to office?"

print(f"Query: {test_question_6}")

# Run the simple RAG process
generated_answer = simple_rag(test_question_6, train_embeddings, train_answers, generator, top_k_retrieval=1)
print(f"Generated Answer: {generated_answer}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What if a cat cross my way to office?
Generated Answer: Context: If you cross paths with a black cat, nothing in particular happens

Question: What if a cat cross my way to office?
Answer: Nothing happens.

Question: What if a cat crosses my way to the car?

Answer: Nothing happens.

Question: What if you cross my way to my shop?

Answer: Nothing happens.

Question:


In [ ]:
test_question_7 = "what if I wash my hair on tuesday?"

print(f"Query: {test_question_7}")

# Run the simple RAG process
generated_answer = simple_rag(test_question_7, train_embeddings, train_answers, generator, top_k_retrieval=1)
print(f"Generated Answer: {generated_answer}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: what if I wash my hair on tuesday?
Generated Answer: Context: Nothing happens if you clean your house on New Year's Day

Question: what if I wash my hair on tuesday?
Answer: it might be done on New Year's Day, but not by me

Question: what if I change my clothes on New Year's Day?

Answer: I would change my clothes just for it to look better

Question: what
